## IMPLEMENTACIÓN DE MODELO DE RECOMENDACIÓN

Para este modelo se usa la tecnica BERT (Bidirectional Encoder Representations from Transformers) y la similitud de coseno para obtener 5 sugerencias de peliculas basadas en su similitud con una en específica. 

In [1]:
#Librerias
import numpy as np
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import joblib

In [2]:
new_data = pd.read_csv("../PI_MLOps/FAST_API/new_data.csv")

Se utilizará la columna "overview" para la vectorizacion y la similitud de coseno

In [4]:
#Se cambia a tipo de dato "str" para evitar errores a futuro
new_data["overview"] = new_data["overview"].astype("str", errors='raise')

In [16]:
#Se convierte en matriz para convertirlos a vectores
X = np.array(new_data["overview"])

In [19]:
#Aplicar el modelo
text_data = X
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embeddings = model.encode(text_data, show_progress_bar=True)

Batches: 100%|██████████| 1420/1420 [2:16:30<00:00,  5.77s/it] 


In [21]:
#Convertirlos a matriz 
embed_data = embeddings
X = np.array(embed_data)

In [ ]:
#Aplicar algoritmo de similitud de coseno
cos_sim_data = pd.DataFrame(cosine_similarity(X))

In [6]:
#Guardar el modelo para ejecutarlo luego
joblib.dump(cos_sim_data, "cos_sim_data.joblib")

['cos_sim_data.joblib']

In [3]:
cos_sim_data = joblib.load("FAST_API/cos_sim_data.joblib")

In [4]:
#Funcion para obtener la recomendacion 
def recomendacion(pelicula):
  movies = new_data["title"] == pelicula
  index = new_data[movies].index[0]
  index_recomm =cos_sim_data.loc[index].sort_values(ascending=False).index.tolist()[1:6]
  movies_recomm =  new_data['title'].loc[index_recomm].values
  return f"Peliculas recomendadas: {movies_recomm}"

In [5]:
recomendacion("Jumanji")

"Peliculas recomendadas: ['Thunder and the House of Magic' 'Aliens in the Attic' 'Kooky'\n 'Ronja Robbersdaughter' 'The Witches']"

Aplicamos la funcion a toda la columna de titulos de peliculas para hacer una nueva columna con las recomendaciones. Este nuevo dataset servirá para la ejecución del FastApi

In [5]:

new_data['recomendaciones'] = new_data['title'].apply(recomendacion)

In [6]:
#Creamos un nuevo dataset
new_data.to_csv("new_data.csv")

In [14]:
new_data[mascara]["recomendaciones"].values[0]

"Peliculas recomendadas: ['Thunder and the House of Magic' 'Aliens in the Attic' 'Kooky'\n 'Ronja Robbersdaughter' 'The Witches']"

Con la siguiente funcion se puede observar la similitud que el modelo puede detectar; se ingresa una pelicula sobre "aliens" y devuelve peliculas con tramas de nuevos planetas, mundos o extraterrestres. 

In [27]:
def give_recommendations(pelicula,print_recommendation,print_recommendation_plots):
  movies = new_data["title"] == pelicula
  index = new_data[movies].index[0]
  index_recomm =cos_sim_data.loc[index].sort_values(ascending=False).index.tolist()[1:6]
  movies_recomm =  new_data['title'].loc[index_recomm].values
  result = {'Movies':movies_recomm,'Index':index_recomm}
  if print_recommendation_plots==True:
    print('La reseña de la pelicula que se ingreso es:\n %s \n'%(new_data['overview'].loc[index]))
    k=1
    for q in range(len(movies_recomm)):
      plot_q = new_data['overview'].loc[index_recomm[q]]
      print('La Pelicula %i sugerida es:\n %s \n'%(k,plot_q))
      k=k+1
  return result

In [28]:
give_recommendations("Aliens in the Attic", True, True)

La reseña de la pelicula que se ingreso es:
 It's summer vacation, but the Pearson family kids are stuck at a boring lake house with their nerdy parents. That is until feisty, little, green aliens crash-land on the roof, with plans to conquer the house AND Earth! Using only their wits, courage and video game-playing skills, the youngsters must band together to defeat the aliens and save the world - but the toughest part might be keeping the whole thing a secret from their parents! Featuring an all-star cast including Ashley Tisdale, Andy Richter, Kevin Nealon, Tim Meadows and Doris Roberts, Aliens In The Attic is the most fun you can have on this planet! 

La Pelicula 1 sugerida es:
 Three teenagers are confined to an isolated country estate that could very well be on another planet. The trio spend their days listening to endless homemade tapes that teach them a whole new vocabulary. Any word that comes from beyond their family abode is instantly assigned a new meaning. Hence 'the sea'

{'Movies': array(['Dogtooth', 'Home Alone: The Holiday Heist',
        'Welcome to the Space Show', 'Jumanji', 'Save the Green Planet!'],
       dtype=object),
 'Index': [15150, 25329, 19978, 1, 9841]}

In [7]:

recomendacion("Aliens in the Attic")

"Peliculas recomendadas: ['Dogtooth' 'Home Alone: The Holiday Heist' 'Welcome to the Space Show'\n 'Jumanji' 'Save the Green Planet!']"